In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from collections import Counter

In [4]:
#Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

In [5]:
#Read the loans data
loans = pd.read_csv('lending-club-data/lending-club-data.csv')

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train_index = pd.read_json('module-5-assignment-2-train-idx.json')
test_index = pd.read_json('module-5-assignment-2-test-idx.json')

In [7]:
train_data = loans.iloc[train_index[0], :]
test_data = loans.iloc[test_index[0], :]

In [8]:
pd.DataFrame(train_index).head()

,0
0,1
1,6
2,7
3,10
4,12


In [9]:
train_data.shape

(37224, 68)

In [10]:
test_data.shape

(9284, 68)

## Features

We will be considering only the following features

In [11]:
[col for col in train_data.columns if(col.startswith('bad'))]

['bad_loans']

In [12]:
#Create a new column named 'safe_loans' using the column 'bad_loans'
def create_safe(data):
    if('bad_loans' in data.columns):
        data['safe_loans'] = data['bad_loans'].apply(lambda x:1 if(x==0) else 0)
    

In [13]:
list(map(create_safe, [train_data, test_data]))

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


[None, None]

In [15]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [16]:
train_data[features + [target]].dtypes

grade             object
term              object
home_ownership    object
emp_length        object
safe_loans         int64
dtype: object

In [17]:
test_data[features + [target]].dtypes

grade             object
term              object
home_ownership    object
emp_length        object
safe_loans         int64
dtype: object

## Missing Value Analysis

In [18]:
def find_missing_values(data):
    features = data.columns
    
    missing_value_count = data.isna().sum()
    
    missing_value_percentage = data.isna().sum()*100/data.shape[0]
    
    missing_data = pd.DataFrame({'Features': features,
                                 'Missing Value Count': missing_value_count,
                                 'Missing Value Percentage': missing_value_percentage},
                                 columns = ['Features', 'Missing Value Count',
                                            'Missing Value Percentage'])
    missing_data = missing_data.sort_values(by='Missing Value Percentage',
                                            ascending = False)
    
    return missing_data

In [19]:
find_missing_values(train_data[features]).head()

,Features,Missing Value Count,Missing Value Percentage
emp_length,emp_length,1443,3.876531
grade,grade,0,0.000000
term,term,0,0.000000
home_ownership,home_ownership,0,0.000000


In [20]:
find_missing_values(test_data[features]).head()

,Features,Missing Value Count,Missing Value Percentage
emp_length,emp_length,349,3.759156
grade,grade,0,0.000000
term,term,0,0.000000
home_ownership,home_ownership,0,0.000000


In [21]:
set(train_data['emp_length'])

{'1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year',
 nan}

In [22]:
#Replace the missing values in the 'emp_length' column with 0
train_data.loc[train_data['emp_length'].isna()==True, 'emp_length']  = '0'

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [23]:
test_data.loc[test_data['emp_length'].isna()==True, 'emp_length'] = '0'

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [24]:
find_missing_values(train_data[features]).head()

,Features,Missing Value Count,Missing Value Percentage
grade,grade,0,0.0
term,term,0,0.0
home_ownership,home_ownership,0,0.0
emp_length,emp_length,0,0.0


In [25]:
find_missing_values(test_data[features]).head()

,Features,Missing Value Count,Missing Value Percentage
grade,grade,0,0.0
term,term,0,0.0
home_ownership,home_ownership,0,0.0
emp_length,emp_length,0,0.0


In [26]:
set(train_data['emp_length'])

{'0',
 '1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year'}

In [27]:
set(test_data['emp_length'])

{'0',
 '1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year'}

## Transform categorical data into binary features

In [28]:
train_data[features].head()

,grade,term,home_ownership,emp_length
1,C,60 months,RENT,< 1 year
6,F,60 months,OWN,4 years
7,B,60 months,RENT,< 1 year
10,C,36 months,RENT,< 1 year
12,B,36 months,RENT,3 years


In [29]:
test_data[features].head()

,grade,term,home_ownership,emp_length
24,D,60 months,RENT,2 years
41,A,36 months,MORTGAGE,10+ years
60,F,60 months,RENT,4 years
93,D,60 months,RENT,10+ years
132,B,36 months,RENT,2 years


In [30]:
OHE_features = []
def to_categorical(data, features = features):
    global OHE_features
    #Convert the categorical features to One-Hot-Encoded features
    df = pd.get_dummies(data[features])
    OHE_features = df.columns.tolist()
    
    #Append the new encoded data frame to the orignal data frame
    data = pd.concat([data, df], axis=1)
    
    #Drop the orignal categorical variables
    if(set(features).intersection() == set(features)):
        data = data.drop(features, axis=1)
    
    return (data)

In [31]:
#train_data.columns.tolist()

In [32]:
train_data, test_data = list(map(to_categorical, [train_data, test_data]))

In [33]:
#Check whether the old features are present in the data
set(train_data.columns).intersection(features) == set() and \
set(test_data.columns).intersection(features) == set()

True

In [34]:
#Check whether the old features are present in the data
set(train_data.columns).intersection(features) != set() or \
set(test_data.columns).intersection(features) != set()

False

In [35]:
train_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,sub_grade,emp_title,annual_inc,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
1,1077430,1314167,2500,2500,2500,15.27,59.83,C4,Ryder,30000.0,...,0,0,0,0,0,0,0,0,0,1
6,1071795,1306957,5600,5600,5600,21.28,152.39,F2,NaN,40000.0,...,0,0,0,1,0,0,0,0,0,0
7,1071570,1306721,5375,5375,5350,12.69,121.45,B5,Starbucks,15000.0,...,0,0,0,0,0,0,0,0,0,1
10,1064687,1298717,9000,9000,9000,13.49,305.38,C1,Va. Dept of Conservation/Recreation,30000.0,...,0,0,0,0,0,0,0,0,0,1
12,1069057,1303503,10000,10000,10000,10.65,325.74,B2,SFMTA,100000.0,...,0,0,1,0,0,0,0,0,0,0


In [36]:
target in test_data.columns

True

In [37]:
binary_features = train_data.columns

In [38]:
target in train_data.columns

True

In [40]:
np.unique(train_data[target])

array([0, 1])

## Calculate the Mistakes

In [46]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node)==0:
        return (0)
    
    #Find the number of -1s
    neg_count = len([ele for ele in labels_in_node if(ele==-1)])
    
    #Find the number of 1s
    pos_count = len([ele for ele in labels_in_node if(ele==1)])
    
    # Return the number of mistakes that the majority classifier makes.
    
    if(neg_count>pos_count):
        max_class = -1
    else:
        max_class = 1
    
    #Get the prediction array
    predicted = np.array(len(labels_in_node) * [max_class])
    
    mistakes = 0
    for act, pred in zip(labels_in_node, predicted):
        if(act!=pred):
            mistakes += 1
    
    return mistakes

In [116]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 1 failed... try again!')

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 3 failed... try again!')
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


## 9. Follow these steps to implement best_splitting_feature:

    Step 1: Loop over each feature in the feature list
    
    Step 2: Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the left split), and one group where all of the data has feature value 1 or True (we will call this the right split). Make sure the left split corresponds with 0 and the right split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
    
    Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula to compute theclassification error.
    
    Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.

In [48]:
def split_on(feature):
    intermediate_node_num_mistakes(labels_in_node)

In [136]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data.loc[data[feature] == 0, feature]
        print("Left Split:-\n")
        print(left_split)
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split = data.loc[data[feature] == 1, feature] 
        print("Right Split:-\n")
        print(right_split)
        
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split)             

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split)
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes)/num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
    
    return best_feature # Return the best feature we found

In [137]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction': None}   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1         ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1        ## YOUR CODE HERE        

    # Return the leaf node
    return (leaf)

We have provided a function that learns the decision tree recursively 
and implements 3 stopping conditions:

    Stopping condition 1: All data points in a node are from the same class.
    
    Stopping condition 2: No more features to split on.
    
    Additional stopping condition: In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the max_depth of the tree. By not letting the tree grow too deep,
    
    we will save computational effort in the learning process.

In [138]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    #remaining_features = remaining_features.tolist()
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth,\
                                                      len(target_values)\
                                                     ))

    # Stopping condition 1
    # (Check if there are mistakes at current node.)
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.
    if(intermediate_node_num_mistakes) == 0:  ## YOUR CODE HERE
        print ("Stopping condition 1 reached.")
        
        # If no mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0:   ## YOUR CODE HERE
        print ("Stopping condition 2 reached.")    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:  ## YOUR CODE HERE
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data, remaining_features,\
                                               target)
    #splitting_feature contains the feature value in the form of string
    
    # Split on the best feature that we found. 
    left_split = data.loc[data[splitting_feature] == 0]
    
    ## YOUR CODE HERE
    right_split = data.loc[data[splitting_feature] == 1]      
    
    remaining_features.remove(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        ## YOUR CODE HERE
        return create_leaf(right_split[target])
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [139]:
OHE_features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_0',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

In [140]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [141]:
small_data_decision_tree = decision_tree_create(train_data, OHE_features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there : 13')

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Left Split:-

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
20       0
21       0
22       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
        ..
37186    0
37187    0
37188    0
37189    0
37191    0
37192    0
37193    0
37194    0
37198    0
37199    0
37200    0
37201    0
37202    0
37203    0
37204    0
37205    0
37206    0
37207    0
37208    0
37211    0
37213    0
37214    0
37215    0
37216    0
37217    0
37218    0
37219    0
37220    0
37222    0
37223    0
Name: grade_A, Length: 32094, dtype: uint8
Right Split:-

19       1
23       1
37       1
40       1
48       1
51       1
55       1
65       1
67       1
73       1
80       1
81       1
85       1
109  

Left Split:-

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
37194    0
37195    0
37196    0
37197    0
37198    0
37199    0
37200    0
37201    0
37202    0
37203    0
37204    0
37205    0
37206    0
37207    0
37208    0
37209    0
37210    0
37211    0
37212    0
37213    0
37214    0
37215    0
37216    0
37217    0
37218    0
37219    0
37220    0
37221    0
37222    0
37223    0
Name: home_ownership_OTHER, Length: 37163, dtype: uint8
Right Split:-

3467     1
3474     1
3731     1
3734     1
3736     1
3768     1
3786     1
3807     1
3888     1
3890     1
3942     1
3946     1
3982     1
4120     1
4420     1
4509     1
4527     1
4562     1
4611     1
4616     1
4620     1
4740     1
11

Left Split:-

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
32       0
        ..
37192    0
37193    0
37194    0
37195    0
37196    0
37197    0
37198    0
37200    0
37201    0
37202    0
37203    0
37204    0
37205    0
37206    0
37207    0
37208    0
37209    0
37210    0
37211    0
37212    0
37213    0
37214    0
37215    0
37216    0
37217    0
37218    0
37219    0
37221    0
37222    0
37223    0
Name: emp_length_5 years, Length: 34347, dtype: uint8
Right Split:-

11       1
12       1
24       1
47       1
52       1
61       1
77       1
91       1
93       1
101      1
110      1
122      1
128      1
130      1
133      1
137      1
146      1
147      1
172      1
178      1
195      1
210      1
211 

Left Split:-

0        0
1        0
2        0
10       0
11       0
13       0
17       0
21       0
32       0
33       0
38       0
39       0
41       0
42       0
43       0
44       0
53       0
55       0
57       0
60       0
67       0
68       0
69       0
72       0
75       0
77       0
83       0
84       0
89       0
94       0
        ..
37109    0
37114    0
37116    0
37119    0
37122    0
37130    0
37132    0
37137    0
37143    0
37144    0
37148    0
37150    0
37156    0
37159    0
37162    0
37164    0
37168    0
37169    0
37174    0
37181    0
37188    0
37192    0
37194    0
37198    0
37199    0
37205    0
37208    0
37214    0
37217    0
37218    0
Name: grade_G, Length: 8884, dtype: uint8
Right Split:-

91       1
214      1
305      1
325      1
333      1
340      1
355      1
369      1
509      1
521      1
649      1
659      1
684      1
697      1
889      1
919      1
933      1
1029     1
1089     1
1127     1
1137     1
1175     1
1185     1
1196 

Left Split:-

4        0
5        0
6        0
8        0
12       0
14       0
15       0
16       0
19       0
20       0
23       0
25       0
26       0
27       0
28       0
29       0
31       0
34       0
35       0
36       0
37       0
40       0
45       0
46       0
47       0
48       0
49       0
50       0
51       0
52       0
        ..
37171    0
37172    0
37173    0
37175    0
37177    0
37178    0
37179    0
37182    0
37184    0
37187    0
37189    0
37190    0
37195    0
37196    0
37197    0
37200    0
37201    0
37202    0
37203    0
37204    0
37206    0
37207    0
37209    0
37210    0
37211    0
37212    0
37213    0
37219    0
37221    0
37223    0
Name: grade_C, Length: 20779, dtype: uint8
Right Split:-

3        1
7        1
9        1
18       1
22       1
24       1
30       1
58       1
59       1
66       1
70       1
74       1
79       1
92       1
96       1
97       1
102      1
103      1
106      1
112      1
119      1
123      1
130      1
131 

Left Split:-

3        0
4        0
6        0
7        0
8        0
9        0
12       0
14       0
15       0
16       0
18       0
20       0
22       0
23       0
24       0
25       0
27       0
28       0
31       0
34       0
35       0
37       0
45       0
47       0
50       0
51       0
52       0
54       0
56       0
58       0
        ..
37160    0
37163    0
37165    0
37166    0
37170    0
37171    0
37175    0
37176    0
37177    0
37178    0
37180    0
37182    0
37185    0
37186    0
37187    0
37190    0
37191    0
37193    0
37195    0
37196    0
37197    0
37207    0
37209    0
37211    0
37213    0
37215    0
37216    0
37219    0
37220    0
37221    0
Name: emp_length_10+ years, Length: 20793, dtype: uint8
Right Split:-

5        1
19       1
26       1
29       1
30       1
36       1
40       1
46       1
48       1
49       1
62       1
66       1
70       1
76       1
81       1
97       1
102      1
104      1
111      1
123      1
131      1
142      1
14

Split on feature home_ownership_RENT. (14665, 13336)
--------------------------------------------------------------------
Subtree, depth = 2 (14665 data points).
Left Split:-

8        0
18       0
22       0
29       0
31       0
52       0
62       0
66       0
82       0
86       0
87       0
88       0
96       0
98       0
101      0
102      0
105      0
106      0
119      0
124      0
130      0
146      0
150      0
151      0
153      0
166      0
180      0
183      0
186      0
197      0
        ..
37140    0
37141    0
37142    0
37145    0
37147    0
37149    0
37151    0
37152    0
37154    0
37157    0
37160    0
37166    0
37167    0
37172    0
37176    0
37179    0
37187    0
37191    0
37200    0
37201    0
37202    0
37204    0
37206    0
37207    0
37211    0
37215    0
37216    0
37219    0
37220    0
37223    0
Name: grade_A, Length: 11460, dtype: uint8
Right Split:-

23       1
40       1
48       1
73       1
80       1
81       1
85       1
120      1
136    

Right Split:-

165      1
227      1
284      1
388      1
437      1
534      1
548      1
616      1
656      1
695      1
696      1
721      1
746      1
783      1
924      1
1036     1
1040     1
1098     1
1118     1
1119     1
1147     1
1161     1
1163     1
1203     1
1236     1
1242     1
1263     1
1272     1
1434     1
1506     1
        ..
36382    1
36391    1
36400    1
36417    1
36462    1
36507    1
36513    1
36519    1
36600    1
36738    1
36763    1
36789    1
36796    1
36808    1
36823    1
36829    1
36847    1
36851    1
36857    1
36875    1
36921    1
36931    1
36950    1
36989    1
37044    1
37045    1
37079    1
37082    1
37131    1
37221    1
Name: emp_length_3 years, Length: 1107, dtype: uint8
Left Split:-

8        0
18       0
22       0
23       0
29       0
31       0
40       0
48       0
52       0
62       0
66       0
73       0
81       0
82       0
86       0
87       0
96       0
98       0
101      0
102      0
105      0
106      0
119  

Left Split:-

3        0
4        0
5        0
6        0
7        0
9        0
12       0
14       0
15       0
16       0
19       0
20       0
24       0
25       0
26       0
27       0
28       0
30       0
34       0
35       0
36       0
37       0
45       0
46       0
47       0
49       0
50       0
51       0
54       0
56       0
        ..
37090    0
37091    0
37106    0
37108    0
37113    0
37121    0
37123    0
37129    0
37134    0
37135    0
37139    0
37146    0
37155    0
37158    0
37163    0
37165    0
37170    0
37171    0
37178    0
37180    0
37182    0
37183    0
37184    0
37185    0
37186    0
37189    0
37193    0
37195    0
37203    0
37213    0
Name: home_ownership_MORTGAGE, Length: 13336, dtype: uint8
Right Split:-

Series([], Name: home_ownership_MORTGAGE, dtype: uint8)
Left Split:-

3        0
4        0
5        0
6        0
7        0
9        0
12       0
14       0
15       0
16       0
19       0
20       0
24       0
25       0
26       0
27    

In [142]:
for fea in features:
    print(fea in train_data.columns)

False
False
False
False


## Build the tree!

12. Train a tree model on the train_data. Limit the depth to 6 (max_depth = 6) to make sure the algorithm doesn't run for too long. Call this tree my_decision_tree. Warning: The tree may take 1-2 minutes to learn.
Making predictions with a decision tree

In [143]:
target

'safe_loans'

In [144]:
target in train_data.columns

True

In [145]:
temp = binary_features[:].copy()

In [146]:
len(OHE_features)

25

In [147]:
binary_features

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'int_rate', 'installment', 'sub_grade', 'emp_title', 'annual_inc',
       'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'not_compliant', 'status',
       'inactive_loans', 'bad_loans', 'emp_length_num', 'grade_num',
       'sub_grade_num', 'delinq_2yrs_zero', 'pub_rec_zero',
    

In [148]:
#train_data[target] = loans.loc[train_index[0], target]
#test_data[target] = loans.iloc[test_index[0], target]
my_decision_tree = decision_tree_create(train_data, OHE_features,\
                                        target, current_depth = 0,\
                                        max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Left Split:-

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
20       0
21       0
22       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
        ..
37186    0
37187    0
37188    0
37189    0
37191    0
37192    0
37193    0
37194    0
37198    0
37199    0
37200    0
37201    0
37202    0
37203    0
37204    0
37205    0
37206    0
37207    0
37208    0
37211    0
37213    0
37214    0
37215    0
37216    0
37217    0
37218    0
37219    0
37220    0
37222    0
37223    0
Name: grade_A, Length: 32094, dtype: uint8
Right Split:-

19       1
23       1
37       1
40       1
48       1
51       1
55       1
65       1
67       1
73       1
80       1
81       1
85       1
109  

Left Split:-

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
9        0
10       0
11       0
12       0
14       0
15       0
16       0
17       0
19       0
20       0
21       0
23       0
24       0
25       0
26       0
27       0
28       0
30       0
34       0
35       0
36       0
37       0
        ..
37157    0
37158    0
37161    0
37162    0
37163    0
37164    0
37165    0
37170    0
37171    0
37174    0
37178    0
37180    0
37182    0
37183    0
37184    0
37185    0
37186    0
37189    0
37193    0
37195    0
37198    0
37199    0
37203    0
37205    0
37207    0
37212    0
37213    0
37214    0
37216    0
37221    0
Name: home_ownership_MORTGAGE, Length: 19846, dtype: uint8
Right Split:-

8        1
13       1
18       1
22       1
29       1
31       1
32       1
33       1
48       1
52       1
62       1
73       1
80       1
81       1
82       1
83       1
84       1
85       1
86       1
88       1
96       1
98       1

Left Split:-

0        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
10       0
11       0
12       0
13       0
14       0
15       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
32       0
        ..
37194    0
37195    0
37196    0
37197    0
37198    0
37199    0
37200    0
37201    0
37202    0
37203    0
37204    0
37205    0
37206    0
37207    0
37208    0
37209    0
37210    0
37211    0
37212    0
37213    0
37214    0
37215    0
37216    0
37217    0
37218    0
37219    0
37220    0
37221    0
37222    0
37223    0
Name: emp_length_4 years, Length: 34593, dtype: uint8
Right Split:-

1        1
9        1
16       1
51       1
80       1
85       1
88       1
89       1
118      1
132      1
159      1
176      1
193      1
203      1
223      1
229      1
239      1
241      1
243      1
259      1
264      1
281      1
287 

Left Split:-

0        0
1        0
2        0
10       0
11       0
13       0
17       0
21       0
32       0
33       0
38       0
39       0
41       0
42       0
43       0
44       0
53       0
55       0
57       0
60       0
67       0
68       0
69       0
72       0
75       0
77       0
83       0
84       0
89       0
91       0
        ..
37114    0
37116    0
37119    0
37122    0
37130    0
37132    0
37137    0
37143    0
37144    0
37148    0
37150    0
37156    0
37159    0
37162    0
37164    0
37168    0
37169    0
37174    0
37181    0
37188    0
37192    0
37194    0
37198    0
37199    0
37205    0
37208    0
37214    0
37217    0
37218    0
37222    0
Name: home_ownership_OTHER, Length: 9220, dtype: uint8
Right Split:-

11965    1
12040    1
30902    1
Name: home_ownership_OTHER, dtype: uint8
Left Split:-

0        0
2        0
10       0
11       0
13       0
17       0
21       0
32       0
33       0
38       0
39       0
41       0
43       0
55       0
57 

Left Split:-

3        0
4        0
5        0
6        0
7        0
8        0
9        0
12       0
14       0
15       0
16       0
18       0
19       0
20       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
34       0
35       0
36       0
37       0
40       0
45       0
        ..
37183    0
37184    0
37185    0
37186    0
37187    0
37189    0
37190    0
37191    0
37193    0
37195    0
37196    0
37197    0
37200    0
37201    0
37202    0
37203    0
37204    0
37206    0
37207    0
37209    0
37210    0
37211    0
37212    0
37213    0
37215    0
37216    0
37219    0
37220    0
37221    0
37223    0
Name: grade_E, Length: 26725, dtype: uint8
Right Split:-

56       1
90       1
110      1
149      1
237      1
250      1
262      1
341      1
351      1
372      1
387      1
430      1
459      1
517      1
672      1
840      1
989      1
1005     1
1008     1
1027     1
1033     1
1087     1
1110     1
1186

...

Left Split:-

19       0
37       0
51       0
56       0
65       0
90       0
109      0
110      0
111      0
134      0
137      0
144      0
148      0
149      0
159      0
161      0
171      0
179      0
190      0
195      0
201      0
218      0
219      0
226      0
237      0
264      0
269      0
275      0
276      0
282      0
        ..
36793    0
36802    0
36803    0
36805    0
36812    0
36819    0
36821    0
36835    0
36842    0
36855    0
36861    0
36884    0
36900    0
36944    0
36958    0
36969    0
36971    0
36977    0
36998    0
37031    0
37043    0
37087    0
37090    0
37106    0
37108    0
37113    0
37146    0
37163    0
37182    0
37195    0
Name: emp_length_6 years, Length: 2630, dtype: uint8
Right Split:-

250      1
554      1
958      1
1091     1
1194     1
1331     1
1387     1
1472     1
1515     1
2078     1
2200     1
2485     1
2490     1
2555     1
3260     1
4008     1
4167     1
4289     1
4523     1
4573     1
4593     1
4638     1
4673 

Left Split:-

3        0
7        0
9        0
24       0
30       0
58       0
59       0
70       0
74       0
79       0
92       0
97       0
112      0
123      0
131      0
140      0
145      0
156      0
168      0
176      0
178      0
181      0
222      0
224      0
280      0
288      0
297      0
302      0
322      0
330      0
        ..
36828    0
36859    0
36865    0
36866    0
36872    0
36883    0
36887    0
36894    0
36911    0
36926    0
36938    0
36953    0
36964    0
36970    0
36982    0
36995    0
36999    0
37008    0
37039    0
37053    0
37062    0
37068    0
37069    0
37091    0
37170    0
37180    0
37183    0
37185    0
37186    0
37193    0
Name: emp_length_7 years, Length: 3472, dtype: uint8
Right Split:-

103      1
338      1
618      1
638      1
1057     1
1217     1
1459     1
2238     1
2397     1
2695     1
3190     1
3272     1
3369     1
3574     1
3668     1
3769     1
3920     1
3940     1
4011     1
4194     1
4307     1
4438     1
4698 

Left Split:-

4        0
5        0
6        0
14       0
16       0
20       0
28       0
34       0
35       0
36       0
45       0
46       0
47       0
49       0
50       0
54       0
71       0
78       0
93       0
99       0
100      0
104      0
114      0
122      0
127      0
132      0
142      0
152      0
160      0
170      0
        ..
36924    0
36927    0
36928    0
36941    0
36945    0
36949    0
36967    0
36992    0
36993    0
37022    0
37035    0
37047    0
37048    0
37049    0
37050    0
37059    0
37067    0
37070    0
37075    0
37078    0
37129    0
37134    0
37135    0
37139    0
37158    0
37171    0
37178    0
37184    0
37189    0
37213    0
Name: home_ownership_MORTGAGE, Length: 4237, dtype: uint8
Right Split:-

Series([], Name: home_ownership_MORTGAGE, dtype: uint8)
Left Split:-

4        0
5        0
6        0
14       0
16       0
20       0
28       0
34       0
35       0
36       0
45       0
46       0
47       0
49       0
50       0
54     

Left Split:-

4        0
6        0
14       0
16       0
20       0
28       0
34       0
35       0
45       0
47       0
50       0
54       0
71       0
78       0
93       0
100      0
114      0
122      0
132      0
152      0
160      0
170      0
175      0
182      0
210      0
212      0
223      0
256      0
266      0
268      0
        ..
36788    0
36801    0
36814    0
36824    0
36845    0
36867    0
36880    0
36897    0
36903    0
36907    0
36922    0
36924    0
36927    0
36928    0
36941    0
36945    0
36992    0
37022    0
37047    0
37048    0
37050    0
37059    0
37075    0
37078    0
37129    0
37134    0
37135    0
37158    0
37178    0
37213    0
Name: emp_length_< 1 year, Length: 2946, dtype: uint8
Right Split:-

99       1
127      1
185      1
215      1
228      1
244      1
316      1
371      1
376      1
445      1
447      1
460      1
471      1
494      1
729      1
825      1
855      1
866      1
1018     1
1153     1
1222     1
1286     1
1402


5        0
36       0
46       0
49       0
104      0
142      0
294      0
335      0
533      0
581      0
687      0
1028     0
1052     0
1056     0
1072     0
1079     0
1235     0
1296     0
1302     0
1348     0
1367     0
1464     0
1516     0
1598     0
1631     0
1635     0
1851     0
1946     0
1956     0
2013     0
        ..
36104    0
36127    0
36168    0
36195    0
36204    0
36206    0
36232    0
36251    0
36348    0
36353    0
36361    0
36425    0
36480    0
36521    0
36525    0
36583    0
36665    0
36673    0
36712    0
36735    0
36749    0
36751    0
36841    0
36949    0
36967    0
36993    0
37035    0
37049    0
37184    0
37189    0
Name: grade_D, Length: 778, dtype: uint8
Right Split:-

Series([], Name: grade_D, dtype: uint8)
Left Split:-

5        0
36       0
46       0
49       0
104      0
142      0
294      0
335      0
533      0
581      0
687      0
1028     0
1052     0
1056     0
1072     0
1079     0
1235     0
1296     0
1302     0
1348     

In [149]:
x = 0
print ("%s" %(x))

0


In [150]:
def classify(tree, x, annotate = False):
    #if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    
    else:
        #split on feature.
        #This is chosen as the best feature to split on
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'],\
                                          split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            ### YOUR CODE HERE
            return classify(tree['right'], x, annotate)
               

###  Now, let's consider the first example of the test set and see what my_decision_tree model predicts for this data point.


In [151]:
test_data[OHE_features].head()

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
0,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [152]:
train_data, test_data = list(map(lambda x:x.reset_index(),\
                                 [train_data, test_data]))


ValueError: cannot insert level_0, already exists

In [ ]:
test_data[OHE_features].head()

In [153]:
#print (test_data.loc[0, OHE_features])
print ('Predicted class: %s ' % classify(my_decision_tree,\
                                         test_data.loc[0, OHE_features]))


Predicted class: 1 


In [154]:
classify(my_decision_tree, test_data.loc[0, OHE_features], annotate=True)

Split on term_ 36 months = 0
At leaf, predicting 1


1

In [155]:
def classify_util(data):
    prediction_test = []
    for i in range(len(data)):
        result = classify(my_decision_tree, data.loc[i, OHE_features],\
                      annotate=True)
        prediction_test.append(result)
    return (prediction_test)

In [156]:
test_data[OHE_features]

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
0,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
6,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
8,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


### Quiz question: What was the feature that my_decision_tree first split on while making the prediction for test_data[0]?

In [157]:
print ("term_ 36 months")

term_ 36 months


### Quiz question: What was the first feature that lead to a right split of test_data[0]?



In [158]:
print ("grade_D")

grade_D


### Quiz question: What was the last feature split on before reaching a leaf node for test_data[0]?

In [159]:
print ("grade_D")

grade_D


## Evaluating your decision tree

16. Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset. Write a function called evaluate_classification_error that takes in as input:

    tree (as described above)
    data (a data frame of data points)

In [160]:
test_data[OHE_features].head()

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
0,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [161]:
classify(my_decision_tree, test_data[OHE_features], annotate=True).tolist()

Split on term_ 36 months = 0       0
1       1
2       0
3       0
4       1
5       0
6       0
7       1
8       1
9       1
10      0
11      1
12      1
13      1
14      1
15      1
16      1
17      0
18      0
19      1
20      1
21      1
22      0
23      0
24      0
25      1
26      1
27      0
28      1
29      0
       ..
9254    1
9255    1
9256    1
9257    1
9258    1
9259    1
9260    1
9261    1
9262    1
9263    1
9264    1
9265    1
9266    1
9267    1
9268    1
9269    1
9270    1
9271    1
9272    1
9273    1
9274    1
9275    0
9276    0
9277    0
9278    1
9279    0
9280    1
9281    1
9282    1
9283    1
Name: term_ 36 months, Length: 9284, dtype: uint8


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [1]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    #prediction = classify(my_decision_tree, test_data[OHE_features], annotate=True)
    prediction = classify_util(data)
    
    # Once you've made the predictions, calculate the classification error and return it
    no_mistakes = len([act for act, pred in zip(data[target], prediction) if(act!=pred)])
    num_data_points = float(len(data))
    classification_error = no_mistakes/num_data_points
    
    return (classification_error)

In [163]:
evaluate_classification_error(my_decision_tree, test_data)

Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 1
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_< 1 year = 0
Split on emp_length_2 years = 1
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
Split on emp_length_10+ years = 0
A

At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_5 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 0
Split on home_ownership_OWN = 1
Split on grade_B = 0
Split on grade_C = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_< 1 year = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_own

Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 1
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_< 1 year = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_< 1 year = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_owne

Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 1
Split on emp_length_10+ years = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 1
At leaf, predicting 1
Split on term_ 36 months

Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_< 1 year = 1
At leaf, predicting 1
Split on term

...

Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 1
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 0
Split on grade_A = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 1
At leaf, predicting 1
Split on term_ 36 months = 0


Split on home_ownership_MORTGAGE = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_5 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 0
Split on home_ownership_OWN = 1
Split on grade_B = 0
Split on grade_C = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 1
Split on emp_length_10+ years = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split

Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_< 1 year = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 0
Split on home_ownership_OWN = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
Split on emp_length_2 years = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 0
Split on home_ownership_OWN = 1
Split on grade_B = 0
Split on grade_C = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership

Split on term_ 36 months = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 0
Split on home_ownership_OWN = 1
Split on grade_B = 0
Split on grade_C = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 1
Split on emp_length_10+ years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 0
Split on grade_A = 1
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 1
Split on emp_length_10+ years = 0
Split on emp_length_5 years = 0
At leaf, predicting 1
Split on term_ 36 months = 1
Split on home_ownership_RENT = 0
Split on home_ownership_MORTGAGE = 1
Split on grade_B = 0
Split on grade_C = 0
Split on grade_A = 1
At leaf, predicting

0.503446790176648

### Printing out a decision stump

18. As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). Here we provide Python code to visualize a decision stump. If you are using different software, make sure your code is analogous to:



In [164]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print ("(leaf, label: %s)" % tree['prediction'])
        return None
    print(split_name)
    split_feature, split_value = split_name.split('_')
    print ('                       %s' %(name))
    print ('         |---------------|----------------|')
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),\
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [165]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

(leaf, label: 1)


In [166]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

TypeError: 'NoneType' object is not subscriptable

## Quiz question: What is the path of the first 3 feature splits considered along the left-most branch of my_decision_tree?

## Quiz question: What is the path of the first 3 feature splits considered along the right-most branch of my_decision_tree?

